In [ ]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [ ]:
### Fixer la version 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [ ]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [ ]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [ ]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [ ]:
# Installation d'obitools V4
cd obitools4
make
cd ..

In [ ]:
# Recherche des chemins des inputs et formatage de l'ouput 
json_path="galaxy_inputs/galaxy_inputs.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file=$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')

echo "Input file path: $input_file"


filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"


output_file="outputs/outputfile.$extension"
format=""


Input file path: galaxy_inputs/wolfdiet_obigrep.fastq
Nom de fichier (sans chemin, sans extension) : wolfdiet_obigrep
Extension détectée : fastq


In [ ]:
# ## Argument hasattrib with a option for the multiples arguments
# # Input key 
# key_hasattrib=$(jq -r '.keyXhasattrib' galaxy_inputs/galaxy_inputs.json)
# #Input value 
# value_hasattrib=$(jq -r '.valueXhasattrib' galaxy_inputs/galaxy_inputs.json)


# # Transformer la chaîne en tableau
# IFS=',' read -r -a value_hasattrib <<< "$value_hasattrib"

# # Construire l'argument final
# hasattrib=()
# for i in "${value_hasattrib[@]}"; do
#     hasattrib+=("-a ${key_hasattrib}=${i}")
# done

In [ ]:
# Variable ObiPCR Mandatory

#forward PATTERN or STRING
forward=$(jq -r '.forward' galaxy_inputs/galaxy_inputs.json)
forward="--forward $forward"
echo "variable forward présente: $forward"

#reverse PATTERN or STRING
reverse=$(jq -r '.reverse' galaxy_inputs/galaxy_inputs.json)
reverse="--reverse $reverse"
echo "variable reverse présente: $reverse"

#Max length INTEGER
maxlen=$(jq -r '.maxlen' galaxy_inputs/galaxy_inputs.json)
maxlen="-L $maxlen"
echo "variable maxlen présente: $maxlen"

# Variable ObiPCR Optional

#allowed mismatches INTEGER 
mismatches=$(jq -r '.mismatches' galaxy_inputs/galaxy_inputs.json)
if [ "$mismatches" != "null" ]; then
    mismatches="-e $mismatches"
    echo "variable mismatches présente: $mismatches"
else
    mismatches=""
    echo "variable mismatches absente"
fi

#min length INTEGER
minlen=$(jq -r '.minlen' galaxy_inputs/galaxy_inputs.json)
if [ "$minlen" != "null" ]; then
    minlen="-l $minlen"
    echo "variable minlen présente: $minlen"
else
    minlen=""
    echo "variable minlen absente"
fi

#circular BOOLEAN
circular=$(jq -r '.circular' galaxy_inputs/galaxy_inputs.json)
if [ "$circular" = "true" ]; then
    circular="-c"
    echo "variable circular présente: $circular"
else
    circular=""
    echo "variable circular absente"
fi

#delta INTEGER -D
delta=$(jq -r '.delta' galaxy_inputs/galaxy_inputs.json)
if [ "$delta" != "null" ]; then
    delta="-D $delta"
    echo "variable delta présente: $delta"
else
    delta=""
    echo "variable delta absente"
fi

#only complete flanking BOOLEAN
onlyflank=$(jq -r '.onlyflank' galaxy_inputs/galaxy_inputs.json)
if [ "$onlyflank" = "true" ]; then
    onlyflank="-f"
    echo "variable onlyflank présente: $onlyflank"
else
    onlyflank="" 
    echo "variable onlyflank absente"
fi

variable predicat présente : -p mode!="joined"


In [ ]:
## Test variable 
primer=$(jq -r '.primer' galaxy_inputs/galaxy_inputs.json)

In [ ]:
./obitools4/build/obipcr $forward $reverse $maxlen $mismatches $minlen $circular $delta $onlyflank --out $output_file --no-order